<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4942f8dd638ff58792baa24443a72567f1bc8da49798769308438a66a915b1ff
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 11:17:42
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.43 C
-------------------
Today PnL: -55.09 K (-0.38%)
Current PnL: -17.30 L (-11.64%)
CY Booked + Current PnL: -5.86 L (-3.94%)
-------------------
Total profit:  1.96 L
Total loss:  -19.26 L
-------------------
Total Booked + Current PnL: 21.12 L (17.36%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 82.82 L (58.0%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,14.68,67.0,H-MC,3.49,198010.0,33340.0,10000.0,0.83,20.25,5.05,26.31,89.0,3.33,1.42,29.85,X40N,NTT,AC
76,TTKPRESTIG,770.00,100.64,48.0,M-SC,3.32,86460.0,-14317.0,14413.0,-0.65,-14.21,16.67,0.09,245.0,-0.99,0.62,13.59,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,67.0,H-MC,8.62,112927.0,12423.0,16679.0,0.69,12.36,14.77,28.96,79.0,0.74,0.81,50.53,MH,ATH,METALS
56,RELIANCE,1533.00,-12.54,54.0,H-LC,4.82,219040.0,8674.0,20108.0,-0.23,4.12,9.18,13.68,37.0,0.43,1.57,21.33,XY25,NTT,REFINERIES
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200294.0,156.0,20730.0,-0.26,0.08,10.35,10.44,4.0,0.01,1.43,5.65,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1239.71,79.0,M-SC,6.33,182259.0,1116.0,44344.0,9.32,0.62,24.33,25.10,235.0,0.03,1.30,46.06,XY24,NTT,HEALTHCARE
31,HINDZINC,730.22,23.40,64.0,M-LC,1.69,201253.0,-3823.0,116385.0,2.12,-1.86,57.83,54.89,52.0,-0.03,1.44,21.78,X5K,ATH,METALS
80,VAIBHAVGBL,521.00,64.19,66.0,H-SC,6.51,142134.0,-40641.0,151714.0,1.89,-22.24,106.74,60.77,125.0,-0.27,1.02,27.30,XR,NTT,JEWELLERY
79,UNITDSPR,1644.00,-11.23,59.0,H-LC,7.62,232613.0,-1531.0,50151.0,1.82,-0.65,21.56,20.77,86.0,-0.03,1.67,6.22,X40N,NTT,BREWERIES
34,ICICIPRULI,790.00,-19.50,53.0,H-MC,2.17,138527.0,2739.0,40007.0,1.73,2.02,28.88,31.48,107.0,0.07,0.99,14.38,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.34,50.0,H-LC,1.43,237375.0,-8776.0,87995.0,-4.26,-3.57,37.07,32.18,16.0,-0.10,1.70,31.19,X200,ATH,IT
53,RAJESHEXPO,518.00,1795.20,53.0,L-SC,2.44,51720.0,-85457.0,85550.0,-3.63,-62.30,165.41,0.07,267.0,-1.00,0.37,28.63,OX40N,NTT,JEWELLERY
73,TCS,4476.75,-27.08,46.0,H-LC,12.24,290046.0,-55910.0,130782.0,-2.64,-16.16,45.09,21.64,1.0,-0.43,2.08,2.75,X40,ATH,IT
39,INFY,2275.00,-17.80,49.0,H-LC,8.18,319670.0,6782.0,164918.0,-2.56,2.17,51.59,54.87,3.0,0.04,2.29,8.96,X40,BTT,IT
83,VOLTAS,1530.00,-0.94,49.0,H-MC,2.76,207810.0,16068.0,21695.0,-2.43,8.38,10.44,19.69,99.0,0.74,1.49,15.75,XY25,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1239.71,79.0,M-SC,6.33,182259.0,1116.0,44344.0,9.32,0.62,24.33,25.1,235.0,0.03,1.30,46.06,XY24,NTT,HEALTHCARE
29,HAVELLS,2069.16,0.34,58.0,H-MC,1.93,248555.0,824.0,74243.0,0.16,0.33,29.87,30.3,92.0,0.01,1.78,13.67,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,94.87,53.0,H-SC,9.32,122769.0,-19710.0,39237.0,-0.70,-13.83,31.96,13.70,163.0,-0.50,0.88,44.90,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,100.64,48.0,M-SC,3.32,86460.0,-14317.0,14413.0,-0.65,-14.21,16.67,0.09,245.0,-0.99,0.62,13.59,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.24,60.0,H-SC,0.83,225540.0,-44127.0,80450.0,-0.50,-16.36,35.67,13.47,138.0,-0.55,1.61,14.08,XY24,NTT,PAINTS
18,CERA,9475.0,-20.76,42.0,H-SC,2.16,144279.0,-31624.0,73640.0,0.14,-17.98,51.04,23.89,149.0,-0.43,1.03,24.60,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-21.02,47.0,H-MC,7.27,106866.0,-24969.0,67197.0,-1.71,-18.94,62.88,32.03,98.0,-0.37,0.77,19.70,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,14.68,67.0,H-MC,3.49,198010.0,33340.0,10000.0,0.83,20.25,5.05,26.31,89.0,3.33,1.42,29.85,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1239.71,79.0,M-SC,6.33,182259.0,1116.0,44344.0,9.32,0.62,24.33,25.10,235.0,0.03,1.30,46.06,XY24,NTT,HEALTHCARE
59,SAIL,228.00,49.16,71.0,M-MC,8.22,235225.0,10263.0,157389.0,0.29,4.56,66.91,74.53,192.0,0.07,1.68,38.29,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,14.68,67.0,H-MC,3.49,198010.0,33340.0,10000.0,0.83,20.25,5.05,26.31,89.0,3.33,1.42,29.85,X40N,NTT,AC
17,CAMS,5211.76,0.38,65.0,H-SC,3.02,114181.0,12177.0,31742.0,-0.56,11.94,27.80,43.06,122.0,0.38,0.82,31.10,X40N,ATH,MISC
29,HAVELLS,2069.16,0.34,58.0,H-MC,1.93,248555.0,824.0,74243.0,0.16,0.33,29.87,30.30,92.0,0.01,1.78,13.67,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,67.0,H-MC,8.62,112927.0,12423.0,16679.0,0.69,12.36,14.77,28.96,79.0,0.74,0.81,50.53,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,50.0,M-SC,0.83,99975.0,8476.0,70272.0,-0.36,9.26,70.29,86.07,223.0,0.12,0.72,45.18,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,68.0,H-MC,6.63,186498.0,6690.0,105726.0,0.66,3.72,56.69,62.52,88.0,0.06,1.34,39.08,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.52,32.0,H-SC,11.35,129845.0,6509.0,127248.0,-0.80,5.28,98.00,108.45,119.0,0.05,0.93,28.81,AR,ATH,MISC
85,WIPRO,420.00,-11.98,48.0,M-LC,5.68,155025.0,4080.0,105370.0,-2.28,2.70,67.97,72.51,53.0,0.04,1.11,8.89,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,HCLTECH,1943.91,-1.08,41.0,H-LC,9.84,225766.0,-16158.0,79424.0,-2.00,-6.68,35.18,26.15,8.0,-0.20,1.62,6.69,X40,ATH,IT
82,VBL,671.64,-16.73,41.0,H-LC,6.82,299988.0,-15854.0,128515.0,-0.90,-5.02,42.84,35.67,5.0,-0.12,2.15,8.14,X40N,ATH,FMCG
73,TCS,4476.75,-27.08,46.0,H-LC,12.24,290046.0,-55910.0,130782.0,-2.64,-16.16,45.09,21.64,1.0,-0.43,2.08,2.75,X40,ATH,IT
5,ASIANPAINT,3465.66,-13.06,47.0,H-LC,7.75,219498.0,-32270.0,85473.0,0.51,-12.82,38.94,21.13,27.0,-0.38,1.57,18.07,X40,ATH,PAINTS
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200294.0,156.0,20730.0,-0.26,0.08,10.35,10.44,4.0,0.01,1.43,5.65,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.34,50.0,H-LC,1.43,237375.0,-8776.0,87995.0,-4.26,-3.57,37.07,32.18,16.0,-0.10,1.70,31.19,X200,ATH,IT
41,ITC,452.0,-37.92,49.0,H-LC,2.25,200294.0,156.0,20730.0,-0.26,0.08,10.35,10.44,4.0,0.01,1.43,5.65,X40,NTT,FMCG
64,SIEMENS,4671.5,2.10,59.0,H-LC,3.56,164175.0,-21920.0,69397.0,-1.27,-11.78,42.27,25.51,15.0,-0.32,1.18,20.98,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-9.24,52.0,H-LC,4.64,279443.0,14017.0,42783.0,-0.03,5.28,15.31,21.40,11.0,0.33,2.00,13.48,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.54,54.0,H-LC,4.82,219040.0,8674.0,20108.0,-0.23,4.12,9.18,13.68,37.0,0.43,1.57,21.33,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.08,46.0,H-LC,12.24,290046.0,-55910.0,130782.0,-2.64,-16.16,45.09,21.64,1.0,-0.43,2.08,2.75,X40,ATH,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200294.0,156.0,20730.0,-0.26,0.08,10.35,10.44,4.0,0.01,1.43,5.65,X40,NTT,FMCG
79,UNITDSPR,1644.00,-11.23,59.0,H-LC,7.62,232613.0,-1531.0,50151.0,1.82,-0.65,21.56,20.77,86.0,-0.03,1.67,6.22,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.08,41.0,H-LC,9.84,225766.0,-16158.0,79424.0,-2.00,-6.68,35.18,26.15,8.0,-0.20,1.62,6.69,X40,ATH,IT
82,VBL,671.64,-16.73,41.0,H-LC,6.82,299988.0,-15854.0,128515.0,-0.90,-5.02,42.84,35.67,5.0,-0.12,2.15,8.14,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,2.10,59.0,H-LC,3.56,164175.0,-21920.0,69397.0,-1.27,-11.78,42.27,25.51,15.0,-0.32,1.18,20.98,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200294.0,156.0,20730.0,-0.26,0.08,10.35,10.44,4.0,0.01,1.43,5.65,X40,NTT,FMCG
19,CIPLA,1795.00,-19.44,52.0,H-LC,5.57,214528.0,10028.0,31385.0,-0.63,4.90,14.63,20.25,10.0,0.32,1.54,14.07,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.06,47.0,H-LC,7.75,219498.0,-32270.0,85473.0,0.51,-12.82,38.94,21.13,27.0,-0.38,1.57,18.07,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.54,54.0,H-LC,4.82,219040.0,8674.0,20108.0,-0.23,4.12,9.18,13.68,37.0,0.43,1.57,21.33,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.08,46.0,H-LC,12.24,290046.0,-55910.0,130782.0,-2.64,-16.16,45.09,21.64,1.0,-0.43,2.08,2.75,X40,ATH,IT
39,INFY,2275.00,-17.80,49.0,H-LC,8.18,319670.0,6782.0,164918.0,-2.56,2.17,51.59,54.87,3.0,0.04,2.29,8.96,X40,BTT,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200294.0,156.0,20730.0,-0.26,0.08,10.35,10.44,4.0,0.01,1.43,5.65,X40,NTT,FMCG
82,VBL,671.64,-16.73,41.0,H-LC,6.82,299988.0,-15854.0,128515.0,-0.90,-5.02,42.84,35.67,5.0,-0.12,2.15,8.14,X40N,ATH,FMCG
1,ABB,7934.00,-37.43,68.0,H-LC,9.38,261326.0,-293.0,119504.0,0.14,-0.11,45.73,45.57,7.0,-0.00,1.87,11.20,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7050.00,66.0,L-SC,5.16,78459.0,-15351.0,91970.0,0.19,-16.36,117.22,81.67,269.0,-0.17,0.56,51.94,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,49.0,L-SC,42.32,83136.0,-30413.0,70466.0,-2.30,-26.78,84.76,35.27,268.0,-0.43,0.60,103.48,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,67.0,M-SC,3.93,85206.0,-17004.0,110393.0,-0.84,-16.64,129.56,91.37,208.0,-0.15,0.61,38.32,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.84,46.0,H-SC,12.32,88580.0,-12383.0,106916.0,-0.87,-12.27,120.70,93.63,148.0,-0.12,0.63,31.94,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.62,94530.0,-3450.0,25050.0,-0.08,-3.52,26.50,22.05,152.0,-0.14,0.68,37.41,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1239.71,79.0,M-SC,6.33,182259.0,1116.0,44344.0,9.32,0.62,24.33,25.10,235.0,0.03,1.30,46.06,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.40,73.0,H-SC,13.46,227113.0,-43648.0,373601.0,-1.53,-16.12,164.50,121.86,133.0,-0.12,1.63,77.67,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,67.0,M-SC,3.93,85206.0,-17004.0,110393.0,-0.84,-16.64,129.56,91.37,208.0,-0.15,0.61,38.32,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,64.19,66.0,H-SC,6.51,142134.0,-40641.0,151714.0,1.89,-22.24,106.74,60.77,125.0,-0.27,1.02,27.30,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3802.70,59.0,M-SC,4.92,121658.0,-25710.0,77520.0,0.27,-17.45,63.72,35.16,236.0,-0.33,0.87,24.26,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.55
1,25,44.12
2,50,75.28


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.91
LC    31.19
MC    23.93
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.87
X40      14.95
X40N     11.69
XR       11.50
AR        8.58
XY25      8.40
OX40N     7.97
X200      1.70
MH        1.69
X5K       1.44
SR        1.24
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.95
H-LC    24.62
H-MC    20.95
M-SC    13.43
M-LC     5.52
M-MC     2.68
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.15,-14.63,77.13
FMCG,12.32,-4.31,41.62
FINANCE,10.22,-14.87,64.99
MISC,7.24,-13.77,77.02
BANKS,6.21,-14.98,75.47
PAINTS,5.73,-15.19,32.36
ELECTRICAL,5.63,-4.85,41.64
INSURANCE,3.74,-5.74,45.26
AC,3.62,4.07,11.99


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3064935.0,22
XR,1306993.0,14
AR,1133706.0,9
X40,761116.0,10
X40N,575298.0,9
OX40N,534505.0,10
XY25,374655.0,6
SR,247190.0,2
X5K,116385.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3609199.0,29
M-SC,1371717.0,17
H-MC,1181485.0,15
H-LC,1091405.0,14
M-LC,378474.0,4
M-MC,303254.0,2
L-SC,247986.0,3
L-MC,60176.0,1
L-LC,37763.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1210501.0      6
M-SC       XY24       785561.0      7
H-SC       AR         781754.0      5
           XR         759614.0      7
H-MC       XY24       548660.0      4
H-LC       X40        481327.0      5
M-MC       XY24       303254.0      2
H-SC       X40N       293255.0      4
           OX40N      254883.0      4
           SR         247190.0      2
H-LC       X40N       210051.0      3
H-MC       X40        207623.0      4
H-LC       AR         188901.0      2
H-MC       XY25       163608.0      2
M-SC       AR         163051.0      2
L-SC       XR         162436.0      2
M-LC       XY24       156719.0      2
M-SC       OX40N      126875.0      4
M-LC       X5K        116385.0      1
M-SC       XR         113671.0      2
           XY25       110393.0      1
H-MC       XR         105726.0      1
M-LC       XR         105370.0      1
H-LC       X200        87995.0      1
L-SC       OX40N       85550.0      1
M-SC       X40         72166.0      1
H-MC       X40N        71992.0      2
           OX40N       67197.0      1
H-LC       XY25        62891.0      2
H-SC       MH          62002.0      1
H-LC       XY24        60240.0      1
L-MC       XR          60176.0      1
L-LC       XY25        37763.0      1
H-MC       MH          16679.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
